<a href="https://colab.research.google.com/github/dibadabir/Project1_A24/blob/main/SkinLesion_PreProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import os
import random
from google.colab import drive
import cv2
import numpy as np
from tqdm import tqdm

# Step 1: Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Step 2: Specify paths
input_folder = "/content/drive/MyDrive/Discipline Specific/HAM 10000"
output_folder = "/content/drive/MyDrive/1st Model/Lesion PreProcessed"

# Check if input and output folders exist
if not os.path.exists(input_folder):
    raise FileNotFoundError(f"The input folder '{input_folder}' does not exist.")
if not os.path.exists(output_folder):
    os.makedirs(output_folder, exist_ok=True)

# Step 3: Select 1,000 random images
image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
selected_images = random.sample(image_files, min(1000, len(image_files)))

print(f"Found {len(image_files)} images in the input folder.")
print(f"Randomly selected {len(selected_images)} images for processing...")

# Step 4: Define the DullRazor hair removal function
def remove_hair_dullrazor(image):
    """Removes hair from the input image using the DullRazor algorithm."""
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply a blackhat morphological operation to highlight hair regions
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (17, 17))
    blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)

    # Use a binary threshold to create a mask for the hair
    _, hair_mask = cv2.threshold(blackhat, 10, 255, cv2.THRESH_BINARY)

    # Apply inpainting to remove hair
    inpainted_image = cv2.inpaint(image, hair_mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)

    return inpainted_image

# Step 5: Define the image processing function
def process_and_save_images(selected_images):
    for image_file in tqdm(selected_images):
        input_path = os.path.join(input_folder, image_file)
        output_path = os.path.join(output_folder, image_file)

        # Load the image
        image = cv2.imread(input_path)

        if image is None:
            print(f"Error reading image: {image_file}. Skipping...")
            continue

        # Step 1: Remove hair using DullRazor algorithm
        image_no_hair = remove_hair_dullrazor(image)

        # Step 2: Resize the image
        resized_image = cv2.resize(image_no_hair, (224, 224))

        # Step 3: Normalize pixel values to the range [0, 1]
        normalized_image = resized_image / 255.0  # Scaling

        # Step 4: Convert back to uint8 format for saving (optional)
        normalized_image_uint8 = (normalized_image * 255).astype(np.uint8)

        # Save the processed image
        cv2.imwrite(output_path, normalized_image_uint8)

# Step 6: Process the images
process_and_save_images(selected_images)

print(f"Processing completed. All images saved to '{output_folder}'.")


Mounted at /content/drive
Found 10015 images in the input folder.
Randomly selected 1000 images for processing...


100%|██████████| 1000/1000 [04:06<00:00,  4.06it/s]

Processing completed. All images saved to '/content/drive/MyDrive/1st Model/Lesion PreProcessed'.
